# Rutina de análisis inicial y de resultados de datos etiquetados durante QC a EMA

> Elaborado por Paola Álvarez, profesional contratista IDEAM, contrato 196 de 2024. Comentarios o inquietudes, remitir a *palvarez@ideam.gov.co* 

El análisis de resultados se incluye dentro del documento de diagnóstico de series temporales.
___
**Librerías:**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
import matplotlib.image as mpimg
import glob
import os
import statistics
from matplotlib.ticker import FuncFormatter
from scipy import stats

_____

### Longitud y continuidad series de datos de EMA - Gráficas

In [3]:
# Path to the directory containing the CSV files
data_directory = "../OE_3_QC_Variables/3_TemperaturaSuelo/0241/RawUnmodified_TS-10cm"
# List to hold dataframes
dataframes = []

# Load each CSV file into a dataframe and add it to the list
for filename in os.listdir(data_directory):
    if filename.endswith(".csv"):
        try:
            df = pd.read_csv(os.path.join(data_directory, filename), parse_dates=['Fecha'], encoding='latin-1')

            # Se verifica si 'Estado' existe y se aplica filtro
            if 'Estado' in df.columns:
                df = df[~df['Estado'].apply(lambda x: any([str(x).startswith(prefix) for prefix in ['0PSO0','0PAT','0PER']]))]

            # Only proceed if the DataFrame is not empty after filtering
            if not df.empty:
                df = df.copy()
                df.set_index('Fecha', inplace=True)
                df['Presence'] = 1  # Add a column to indicate data presence
                dataframes.append(df)
        except Exception as e:
            print(f"Error loading {filename}: {e}")

# Se muestran los headers y fechas por archivo para ver estructura - Para caso de ejemplo
dataframes_info = [(df.head(), df.index.min(), df.index.max()) for df in dataframes]
dataframes_info

[(                     Unnamed: 0   Station               Name  Sensor   Valor  \
  Fecha                                                                          
  2006-01-01 05:59:05      114498  11105020  CARMEN DEL DARIEN     241  2853.0   
  2006-01-01 06:59:05      114499  11105020  CARMEN DEL DARIEN     241   830.1   
  2006-01-01 07:59:05      114500  11105020  CARMEN DEL DARIEN     241  2597.1   
  2006-01-01 08:59:05      114501  11105020  CARMEN DEL DARIEN     241  2955.0   
  2006-01-01 09:59:05      114502  11105020  CARMEN DEL DARIEN     241  2929.5   
  
                       Presence  
  Fecha                          
  2006-01-01 05:59:05         1  
  2006-01-01 06:59:05         1  
  2006-01-01 07:59:05         1  
  2006-01-01 08:59:05         1  
  2006-01-01 09:59:05         1  ,
  Timestamp('2006-01-01 05:59:05'),
  Timestamp('2019-07-29 06:27:00')),
 (                     Unnamed: 0   Station    Name  Sensor  Valor  Presence
  Fecha                           

In [28]:
# Esta si
# Ordenar lista de dataframes
dataframes = sorted(dataframes, key=lambda x: x['Station'].iloc[0])

# Se crea el date range con las fechas inicial y final conocidas, para presión atmosférica 01/01/2001 a 03/04/2024, horaria
date_range = pd.date_range(start="2006-01-01 00:00", end="2024-10-15 23:00", freq='h')

# Número de estaciones por gráfico
estaciones_por_grafico = 50

# Total de gráficos a generar
total_graficos = len(dataframes) // estaciones_por_grafico + (len(dataframes) % estaciones_por_grafico > 0)

# Configuración de la fuente para todo el gráfico
font = {'family': 'Franklin Gothic Book',
        'weight': 'normal',
        'size': 10}

plt.rc('font', **font)

for j in range(total_graficos):
    fig, ax = plt.subplots(figsize=(15, 10))
    inicio = j * estaciones_por_grafico
    fin = min(inicio + estaciones_por_grafico, len(dataframes))  # Asegurarse de no pasarse del rango

    # Define a helper function to plot data for clarity
    def plot_station_data(ax, data, index, station_label):
        # Find where data is present
        presence_mask = data['Presence'] == 1
        # Plot only where data is present
        ax.fill_between(data.index, index - 0.45, index + 0.45, where=~presence_mask, color='#f0f0f0', step='mid', label='Sin dato' if i == 1 else "")
        ax.fill_between(data.index, index - 0.45, index + 0.45, where=presence_mask, color='#a04000', step='mid', label='Con dato' if i == 1 else "")

    for i, df in enumerate(dataframes[inicio:fin], start=1):
        # Check for duplicates
        duplicates = df.index.duplicated()
        # Optionally, drop or keep the first/last occurrence of duplicates
        if duplicates.any():
            df = df.loc[~duplicates]
        # Reindex the dataframe to the full date range, filling missing data with 0 (indicating absence)
        df_complete = df.reindex(date_range, fill_value=0)
        plot_station_data(ax, df_complete, i, f"Station {df['Station'].iloc[0]}")

    #ax.legend(loc='upper left')  # Agrega la leyenda en la esquina superior izquierda
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), fancybox=True, shadow=True, ncol=2)
    # Setting the labels and ticks for better readability
    ax.set_yticks(range(1, fin - inicio + 1))
    ax.set_yticklabels([f"{df['Station'].iloc[0]}" for df in dataframes[inicio:fin]], fontsize=8)
    ax.set_ylim(0.5, fin - inicio + 0.5)
    ax.set_xlim(date_range[0], date_range[-1])
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_minor_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    plt.xticks(rotation=90)
    plt.ylabel('Código de estación CNE')
    plt.title(f'Completitud de datos por estación - Grupo de estaciones {j+1} de {total_graficos}')
    plt.grid(True, which='both', linestyle='-', linewidth=0.25)

    # Show and save the plot
    plt.tight_layout()
    plt.savefig(f'completitud_datos_grafico_{j+1}.png')  # Guarda el gráfico como un archivo PNG
    plt.close(fig)  # Cierra la figura para liberar memoria

In [18]:
from matplotlib import font_manager
fonts = [f.name for f in font_manager.fontManager.ttflist]
print(set(fonts))

{'Franklin Gothic Demi', 'Cambria', 'Lucida Sans Unicode', 'Stencil', 'Niagara Engraved', 'Harlow Solid Italic', 'Calibri', 'ESRI AMFM Water', 'Haettenschweiler', 'Microsoft PhagsPa', 'ESRI Meteorological 01', 'Curlz MT', 'Oswald', 'Century', 'DejaVu Sans Display', 'cmex10', 'Microsoft Tai Le', 'Chiller', 'Gigi', 'Maiandra GD', 'ESRI Pipeline US 1', 'Bradley Hand ITC', 'Edwardian Script ITC', 'SimSun-ExtB', 'HoloLens MDL2 Assets', 'ESRI IGL Font16', 'High Tower Text', 'ESRI SDS 1.95 1', 'Pristina', 'Myanmar Text', 'MingLiU-ExtB', 'ESRI Telecom', 'Lucida Handwriting', 'Harrington', 'Rockwell Extra Bold', 'Palace Script MT', 'Vivaldi', 'ESRI MilSym 03', 'Trebuchet MS', 'ESRI AMFM Electric', 'MT Extra', 'ESRI Weather', 'Playbill', 'ESRI IGL Font23', 'ESRI US MUTCD 3', 'Imprint MT Shadow', 'Segoe UI Emoji', 'DejaVu Serif Display', 'Perpetua', 'ESRI Commodities', 'Rockwell', 'ESRI Surveyor', 'Franklin Gothic Heavy', 'Dosis', 'Franklin Gothic Demi Cond', 'STIXSizeFiveSym', 'Tw Cen MT Condens

In [30]:
## Analizar presencia de '0PSO0'
# Se lee la carpeta del ejemplo
# Directorio de ejemplo
directorio = "../OE_3_QC_Variables/3_TemperaturaSuelo/0241/RawUnmodified_TS-10cm" #r'../../OE_3_QC_Variables/2_HumedadRelativa/RawUnmodified_HR/proc_freqs'

# Lista para almacenar los nombres de los archivos y conteos de '0PSO'
archivos_con_0PSO = []

# Iterar sobre cada archivo en el directorio
for archivo in os.listdir(directorio):
    if archivo.endswith('.csv'):
        # Construir la ruta completa al archivo
        ruta_archivo = os.path.join(directorio, archivo)
        
        # Leer el archivo CSV en un DataFrame de pandas
        tipos_de_dato = {'Estado': str}#, 'Estado_Anterior': str}
        df = pd.read_csv(ruta_archivo, dtype=tipos_de_dato, encoding='latin-1')
        
        # Contar las ocurrencias de '0PSO' en la columna 'Estado'
        if 'Estado' in df.columns:
            conteo_0PSO = df['Estado'].value_counts().get('0PSO0', 0)
        else:
            pass
        
        # Si el conteo es mayor que cero, agregar el archivo y el conteo a la lista
        if conteo_0PSO > 0:
            archivos_con_0PSO.append((archivo, conteo_0PSO))

# Imprimir los nombres de los archivos y los conteos de '0PSO' en la columna 'Estado'
print('Archivos que contienen "0PSO" en la columna "Estado" y su conteo:')
for archivo, conteo in archivos_con_0PSO:
    print(f'{archivo}: {conteo} veces')

NameError: name 'conteo_0PSO' is not defined

In [ ]:
# Año-Mes
# Configuración de estilos para el heatmap
sns.set_theme()

def process_file(file_path):
    # Leer el archivo CSV
    df = pd.read_csv(file_path, parse_dates=['Fecha'], encoding='latin-1')
    
    # Extraer el año y el mes de la fecha
    df['Año-Mes'] = df['Fecha'].dt.strftime('%Y-%m')
    
    # Agrupar por 'Año-Mes' y contar registros
    df_grouped = df.groupby('Año-Mes').size().reset_index(name='Count')
    
    return df_grouped

# Ruta de la carpeta con los archivos CSV
folder_path = '"../OE_3_QC_Variables/3_TemperaturaSuelo/0241/RawUnmodified_TS-10cm"'

# Diccionario para guardar datos procesados de cada archivo
data_dict = {}

# Para cada archivo en la carpeta, procesar y agregar al diccionario
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        station_code = pd.read_csv(os.path.join(folder_path, file_name))['Station'].iloc[0]
        df_temp = process_file(os.path.join(folder_path, file_name))
        df_temp['Station'] = station_code
        data_dict[station_code] = df_temp

# Combinar todos los DataFrames en uno solo
final_df = pd.concat(data_dict.values())

# Crear una tabla pivot para el heatmap
pivot_table = final_df.pivot_table(index='Station', columns='Año-Mes', values='Count', aggfunc='sum').fillna(0)

# Crear el heatmap
plt.figure(figsize=(20, len(data_dict)))
sns.heatmap(pivot_table, cmap="Blues", cbar=True, yticklabels=True, xticklabels=True)
plt.title(f"Longitud y continuidad de todas las estaciones")
plt.ylabel("Estación")
plt.xlabel("Año-Mes")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Solo año
# Configuración de estilos para el heatmap
sns.set_theme()

def process_file(file_path):
    # Leer el archivo CSV
    df = pd.read_csv(file_path, parse_dates=['Fecha'], encoding='latin-1')
    
    # Extraer el año y el mes de la fecha
    df['Año'] = df['Fecha'].dt.strftime('%Y')
    
    # Agrupar por 'Año-Mes' y contar registros
    df_grouped = df.groupby('Año').size().reset_index(name='Count')
    
    return df_grouped

# Ruta de la carpeta con los archivos CSV
folder_path = 'RawUnmodified_Patm'

# Diccionario para guardar datos procesados de cada archivo
data_dict = {}

# Para cada archivo en la carpeta, procesar y agregar al diccionario
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        station_code = pd.read_csv(os.path.join(folder_path, file_name), encoding='latin-1')['Station'].iloc[0]
        df_temp = process_file(os.path.join(folder_path, file_name))
        df_temp['Station'] = station_code
        data_dict[station_code] = df_temp

# Combinar todos los DataFrames en uno solo
final_df = pd.concat(data_dict.values())

# Crear una tabla pivot para el heatmap
pivot_table = final_df.pivot_table(index='Station', columns='Año', values='Count', aggfunc='sum').fillna(0)

# Crear el heatmap
# Ajustar el tamaño de la figura
plt.figure(figsize=(20, 0.25 * len(data_dict)))
sns.heatmap(pivot_table, cmap="Blues", cbar=True, yticklabels=True, xticklabels=True)
plt.title(f"Longitud y cantidad anual de datos por estaciones")
plt.ylabel("Estación")
plt.xlabel("Año")
plt.yticks(rotation=0)
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [5]:
ema_df = pd.read_csv('EMA_Patm_LocAOInstitFreq.csv', delimiter=';', encoding='latin-1')
ema_df.head()

,OBJECTID,Station,lat,long_,nombre,sede,id_rule,codigo,Instituc,FreqInf
0,1,11030010,5.375,-76.613,Valle,09 - Cali,9,9,IDEAM,H
1,2,11035030,5.285,-76.628,Valle,09 - Cali,9,9,IDEAM,H
2,3,11045010,5.691,-76.644,Antioquia,01 - Medellin,1,1,IDEAM,H
3,4,11050020,5.995,-76.780,Antioquia,01 - Medellin,1,1,IDEAM,H
4,5,11080010,6.559,-76.885,Antioquia,01 - Medellin,1,1,IDEAM,H


In [ ]:
ema_df['Station'] = ema_df['Station'].str.split(',').str[0].astype('int64')

In [ ]:
ema_df['Station']

In [ ]:
ema_df['AO_Cod'] = ema_df['AO_Cod'].str.split(',').str[0]
ema_df['AO_Cod'] = pd.to_numeric(ema_df['AO_Cod'], errors='coerce')

In [ ]:
ema_df.to_csv('EMA_LimSup_RP_ER_Alt_CDTGr_AO.txt', sep=';', encoding='latin-1')

In [4]:
### Hacer heatmap de las series de datos
# Configuración de estilos para el heatmap
# Función para procesar cada archivo CSV
def process_file(file_path):
    # Leer el archivo CSV
    df = pd.read_csv(file_path, parse_dates=['Fecha'], encoding='latin-1')
    df['Station'] = df['Station'].astype('float64')
    
    # Extraer el año y el mes de la fecha
    df['Año'] = df['Fecha'].dt.year
    df['Mes'] = df['Fecha'].dt.month
    
    # Agrupar por 'Año-Mes' y contar registros
    df_grouped = df.groupby(['Año', 'Mes']).size().reset_index(name='Count')
    df_grouped['Station'] = pd.read_csv(file_path, encoding='latin-1')['Station'].iloc[0]
    
    return df_grouped

# Leer el archivo .txt
ema_df = pd.read_csv('EMA_Patm_LocAOInstitFreq.csv', delimiter=';', encoding='latin-1') # Se espera EMA_AllInfor_Patm'

# Ruta de la carpeta con los archivos CSV
folder_path = 'RawUnmodified_Patm'

# Diccionario para guardar datos procesados de cada archivo
data_dict = {}

# Para cada archivo en la carpeta, procesar y agregar al diccionario
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        station_code = pd.read_csv(os.path.join(folder_path, file_name), encoding='latin-1')['Station'].iloc[0]
        df_temp = process_file(os.path.join(folder_path, file_name))
        df_temp['Station'] = station_code
        data_dict[station_code] = df_temp

# Combinar todos los DataFrames en uno solo
final_df = pd.concat(data_dict.values())

In [ ]:
## Primera forma de graficar
# Merge final_df con ema_df para obtener AO_Cod para cada Station
merged_df = final_df.merge(ema_df[['Station', 'AO_Cod']], on='Station')

# Agrupar estaciones según 'AO_Cod'
grouped = merged_df.groupby('AO_Cod')['Station'].unique()

# Establecer estilo global
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.stretch'] = 'condensed'

# Para cada grupo de estaciones, hacer una gráfica
for ao_cod, stations, projects in grouped.items():
    try:
        ao_cod = int(ao_cod)  # Intentar convertir ao_cod a entero
    except (ValueError, TypeError):
        print(f"No se pudo convertir {ao_cod} a entero. Continuando con el siguiente grupo.")
        continue  # Continuar con la siguiente iteración del bucle
    df_group = merged_df[merged_df['Station'].isin(stations)]
    
    # Crear una tabla pivot sumando los registros por año
    pivot_table = df_group.pivot_table(index='Station', columns='Año', values='Count', aggfunc='sum').fillna(0)
    
    # Crear el heatmap
    plt.figure(figsize=(20, 0.25 * len(stations)))
    ax = sns.heatmap(pivot_table, cmap="Blues", cbar=True, yticklabels=True, xticklabels=True, 
                     cbar_kws={"format": '{:,}', "label": "Cantidad de registros"})

    # Ajustar la apariencia de los ticks de la colorbar
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=15, labelrotation=0)  # Cambia '12' al tamaño de fuente deseado
    cbar.ax.yaxis.set_major_formatter(FuncFormatter(lambda x, _: '{:,.0f}'.format(x)))
    
    # Ajustes del plot
    plt.title(f"Longitud y cantidad de registros de EMA, AO {ao_cod}", 
              fontsize=18, fontweight='bold')
    plt.ylabel("Estación", fontsize=16)
    plt.xlabel("Año", fontsize=16)
    plt.yticks(rotation=0, fontsize=13)
    plt.xticks(rotation=0, fontsize=14)
    plt.tight_layout()
    plt.show()

In [ ]:
## Segunda forma de graficar
# Merge final_df con ema_df para obtener AO_Cod y project para cada Station
ema_df = pd.read_csv('EMA_Patm_LocAOInstitFreq.csv', delimiter=';', encoding='latin-1')
merged_df = final_df.merge(ema_df[['Station', 'AO_Cod', 'Instituc']], on='Station')

# Establecer estilo global
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.stretch'] = 'condensed'

# Si el proyecto es 'IDEAM', se agrupa por AO_Cod. De lo contrario, se agrupa solo por proyecto.
for project, project_group in merged_df.groupby('Instituc'):
    if project == 'IDEAM':
        for ao_cod, ao_group in project_group.groupby('id_rule'):
            try:
                ao_cod = int(ao_cod) # Se intenta convertir ao_cod a entero
            except (ValueError, TypeError):
                print(f"No se pudo convertir {ao_cod} a entero. Continuando con el siguiente grupo.")
                continue  # Continuar con la siguiente iteración del bucle 
            pivot_table = ao_group.pivot_table(index='Station', columns='Año', values='Count', aggfunc='sum').fillna(0)
            pivot_table.to_csv(f'CantRegistros_{project}_AO{ao_cod}.csv')
            plt.figure(figsize=(16, 0.25 * len(ao_group['Station'].unique())))
            ax = sns.heatmap(pivot_table, cmap="Blues", cbar=True, yticklabels=True, xticklabels=True, 
                             cbar_kws={"format": '{:,}', "label": "Cantidad de registros"})
            # Se ajusta la apariencia de los ticks de la colorbar
            cbar = ax.collections[0].colorbar
            cbar.ax.tick_params(labelsize=15, labelrotation=0)  # Cambia '12' al tamaño de fuente deseado
            cbar.ax.yaxis.set_major_formatter(FuncFormatter(lambda x, _: '{:,.0f}'.format(x)))
            
            # Se ajusta  apariencia del plot
            plt.title(f"Longitud y cantidad de registros de EMA, AO {ao_cod}", fontsize=18, fontweight='bold')
            plt.ylabel("Estación", fontsize=16)
            plt.xlabel("Año", fontsize=16)
            plt.yticks(rotation=0, fontsize=13)
            plt.xticks(rotation=0, fontsize=14)
            plt.tight_layout()
            plt.show()
            
    else:
        pivot_table = project_group.pivot_table(index='Station', columns='Año', values='Count', aggfunc='sum').fillna(0)
        pivot_table.to_csv(f'CantRegistros_{project}.csv')
        plt.figure(figsize=(16, 0.25 * len(project_group['Station'].unique())))
        ax = sns.heatmap(pivot_table, cmap='Greens', cbar=True, yticklabels=True, xticklabels=True,
                    cbar_kws={"format": '{:,}', "label": "Cantidad de registros"})
        # Se ajusta la apariencia de los ticks de la colorbar
        cbar = ax.collections[0].colorbar
        cbar.ax.tick_params(labelsize=15, labelrotation=0)  # Cambia '12' al tamaño de fuente deseado
        cbar.set_label("Cantidad de registros", size=15)  
        cbar.ax.yaxis.set_major_formatter(FuncFormatter(lambda x, _: '{:,.0f}'.format(x)))
        
        # Se ajusta  apariencia del plot
        plt.title(f"Longitud y cantidad de datos del proyecto: {project}", fontsize=18, fontweight='bold')
        plt.ylabel("Estación", fontsize=16)
        plt.xlabel("Año", fontsize=16)
        plt.yticks(rotation=0, fontsize=13)
        plt.xticks(rotation=0, fontsize=14)
        plt.tight_layout()
        plt.show()

### Cantidad total de datos

In [7]:
## Se indaga sobre cuál es la cantidad de datos de todas las series descargadas en el período 2001-01-01 al 2024-03-31
# Se crea la función cantidad_datos
def cantidad_datos(archivos):
    total_filas = 0

    for archivo in archivos:
        # Se utiliza el método 'chunksize' de pandas para leer los archivos por partes (chunks)
        # y procesarlos de forma incremental sin cargar todos los datos en memoria al mismo tiempo
        chunks = pd.read_csv(archivo, encoding='latin-1', chunksize=100000)  # Se establece este valor de chunk para poder hacer otras actividades

        for chunk in chunks:
            total_filas += len(chunk)

    return total_filas

# Ruta de la carpeta con los archivos CSV
carpeta_proces = "../OE_3_QC_Variables/3_TemperaturaSuelo/0241/RawUnmodified_TS-10cm/"

# Obtener la lista de archivos en la carpeta
archivos = [carpeta_proces + archivo for archivo in os.listdir(carpeta_proces) if archivo.endswith('.csv')]

# Llamar a la función para contar las filas
total_filas = cantidad_datos(archivos)

# Imprimir el resultado
print("El número total de datos es:", total_filas)

El número total de datos es: 6765154


### Contar datos por fecha

In [9]:
def contar_datos_por_fechas(archivos):
    resultados = []

    for archivo in archivos:
        chunks = pd.read_csv(archivo, encoding='latin-1', chunksize=100000)
        
        for chunk in chunks:
            chunk['Fecha'] = pd.to_datetime(chunk['Fecha'], format='%Y-%m-%d %H:%M:%S.%f')
            estacion = chunk['Station'].iloc[0]
            
            # Conteo por año, mes y día
            conteo_dia = chunk['Fecha'].dt.to_period('D').value_counts().rename_axis('Año-mes-dia').reset_index(name='Cantidad_año-mes-dia')
            conteo_dia['Año-mes'] = conteo_dia['Año-mes-dia'].dt.to_timestamp().dt.to_period('M')
            conteo_dia['Año'] = conteo_dia['Año-mes'].dt.year

            # Conteo por año y mes
            conteo_mes = chunk['Fecha'].dt.to_period('M').value_counts().rename_axis('Año-mes').reset_index(name='Cantidad_año-mes')
            conteo_mes['Año'] = conteo_mes['Año-mes'].dt.year

            # Conteo por año
            conteo_anual = chunk['Fecha'].dt.year.value_counts().rename_axis('Año').reset_index(name='Cantidad_año')
            
            # Combinar con la información de la estación
            conteo_anual['Station'] = estacion
            conteo_mes['Station'] = estacion
            conteo_dia['Station'] = estacion
            
            # Merge considerando Station, Año y Año-mes
            merged_data = pd.merge(pd.merge(conteo_anual, conteo_mes, on=['Station', 'Año']), conteo_dia, on=['Station', 'Año', 'Año-mes'])
            resultados.append(merged_data)
    
    df_resultados = pd.concat(resultados, ignore_index=True)
    # Reordenar columnas
    df_resultados = df_resultados[['Station', 'Año', 'Cantidad_año', 'Año-mes', 'Cantidad_año-mes','Año-mes-dia','Cantidad_año-mes-dia']]
    df_resultados.sort_values(by=['Station', 'Año', 'Año-mes', 'Año-mes-dia'], inplace=True)
    
    # Rellenar la columna 'Cantidad_año' sólo una vez por cada año y estación
    df_resultados['Cantidad_año'] = df_resultados.groupby(['Station', 'Año'])['Cantidad_año'].transform(lambda x: x.iloc[0] if not x.isna().all() else pd.NA)
    
    df_resultados.to_csv('conteo_datos_detalles_completo.csv', index=False, encoding='latin-1')
    return df_resultados

# Uso de la función
carpeta_proces = "../OE_3_QC_Variables/3_TemperaturaSuelo/0241/RawUnmodified_TS-10cm/"
archivos = [carpeta_proces + archivo for archivo in os.listdir(carpeta_proces) if archivo.endswith('.csv')]
resultados = contar_datos_por_fechas(archivos)
print(resultados.head())

      Station   Año  Cantidad_año  Año-mes  Cantidad_año-mes Año-mes-dia  \
150  11105020  2006         12391  2006-01              1072  2006-01-01   
128  11105020  2006         12391  2006-01              1072  2006-01-02   
140  11105020  2006         12391  2006-01              1072  2006-01-03   
139  11105020  2006         12391  2006-01              1072  2006-01-04   
138  11105020  2006         12391  2006-01              1072  2006-01-05   

     Cantidad_año-mes-dia  
150                    19  
128                    24  
140                    24  
139                    24  
138                    24  


In [7]:
### Lectura de frecuencias para revisión inicial
dffreq = pd.read_csv('../../OE_2_DiagnosticoActualDatos/An8_ConteoDatos_AnioMesDia_HR.csv', encoding='latin-1')
dffreq

,Station,Año,Cantidad_año,Año-mes,Cantidad_año-mes,Año-mes-dia,Cantidad_año-mes-dia
0,11025501,2016,32894,2016-06,7589,2016-06-02,113
1,11025501,2016,32894,2016-06,7589,2016-06-03,204
2,11025501,2016,32894,2016-06,7589,2016-06-05,84
3,11025501,2016,32894,2016-06,7589,2016-06-06,288
4,11025501,2016,32894,2016-06,7589,2016-06-07,288
...,...,...,...,...,...,...,...
1293409,5311500149,2024,8244,2024-03,622,2024-03-01,138
1293410,5311500149,2024,8244,2024-03,622,2024-03-02,144
1293411,5311500149,2024,8244,2024-03,622,2024-03-03,144
1293412,5311500149,2024,8244,2024-03,622,2024-03-04,144


In [8]:
grupito = dffreq.groupby(dffreq['Station'])

In [10]:
grp_mx = grupito['Cantidad_año'].max()
grp_mx.to_csv('grp_mx.csv', encoding='latin-1', index=True, sep=';')

### Obtener fechas iniciales y finales de estaciones

In [11]:
def obtener_fechas(archivo):
    try:
        # Se lee el archivo
        datos = pd.read_csv(archivo, encoding='latin-1')#, names=['Fecha', 'Valor'])
        try:
            datos['Fecha'] = pd.to_datetime(datos['Fecha'], format='%Y-%m-%d %H:%M:%S.%f')
        except ValueError:
            datos['Fecha'] = pd.to_datetime(datos['Fecha'], format='%Y-%m-%d %H:%M:%S')
        
        # Se obtiene el código de la estación
        station = datos['Station'].values[0]
        
        # Se obtiene primera y última fecha 
        fecha_inicial = datos['Fecha'].iloc[0]
        fecha_final = datos['Fecha'].iloc[-1]
        
        return station, fecha_inicial, fecha_final
    
    except Exception as e:
        print(f"Error con el archivo {archivo}: {e}")
        return None, None, None

def main():
    # Cambia la ruta según la ubicación de tu carpeta
    ruta_carpeta = "../OE_3_QC_Variables/3_TemperaturaSuelo/0241/RawUnmodified_TS-10cm/"

    # Listamos todos los archivos en la carpeta que terminen con .csv
    archivos = [f for f in os.listdir(ruta_carpeta) if f.endswith('.csv')]
    
    # Creamos una lista para almacenar los resultados
    resultados = []

    # Iteramos sobre cada archivo y obtenemos las fechas
    for archivo in archivos:
        ruta_archivo = os.path.join(ruta_carpeta, archivo)
        station, fecha_inicial, fecha_final = obtener_fechas(ruta_archivo)
        
        if station and fecha_inicial and fecha_final:
            resultados.append([station, fecha_inicial, fecha_final])
    
    # Convertimos los resultados a un DataFrame
    resultados_df = pd.DataFrame(resultados, columns=['CodEstacion', 'fecha_inicial', 'fecha_final'])
    
    # Guardamos el DataFrame como un archivo CSV
    resultados_df.to_csv('FechaInicialFinal_TS10.csv', index=False)

if __name__ == "__main__":
    main()

### Obtener estadísticos descriptivos

In [13]:
def obtener_EstadDescript(archivo):
    try:
        # Se lee el archivo
        datos = pd.read_csv(archivo, encoding='latin-1')#, names=['Fecha', 'Valor'])
        try:
            datos['Fecha'] = pd.to_datetime(datos['Fecha'], format='%Y-%m-%d %H:%M:%S.%f')
        except ValueError:
            datos['Fecha'] = pd.to_datetime(datos['Fecha'], format='%Y-%m-%d %H:%M:%S')
        
        # Se obtiene el código de la estación
        station = datos['Station'].values[0]
        
        # Se obtienen los estadísticos descriptivos
        minimo = datos['Valor'].min()
        maximo = datos['Valor'].max()
        media = datos['Valor'].mean()
        mediana = datos['Valor'].median()
        desvest = datos['Valor'].std()
        varianza = datos['Valor'].var()
        #first_q = datos['Valor'].quantile(0.25)
        #third_q = datos['Valor'].quantile(0.75)
        
        return station, minimo, maximo, media, mediana, desvest, varianza, #first_q, third_q, mediana --no fue posible calcular estos estad.
    
    except Exception as e:
        print(f"Error con el archivo {archivo}: {e}")
        print(e.__class__)
        return None, None, None, None, None, None, None#, None, None --no fue posible calcular estos estad.

def main():
    # Cambia la ruta según la ubicación de tu carpeta
    ruta_carpeta = "../OE_3_QC_Variables/3_TemperaturaSuelo/0241/RawUnmodified_TS-10cm/"

    # Listamos todos los archivos en la carpeta que terminen con .csv
    archivos = [f for f in os.listdir(ruta_carpeta) if f.endswith('.csv')]
    
    # Se crea una lista para almacenar los resultados
    resultados = []

    # Se itera sobre cada archivo y se obtienen fechas las fechas
    for archivo in archivos:
        ruta_archivo = os.path.join(ruta_carpeta, archivo)
        station, minimo, maximo, media, mediana, desvest, varianza= obtener_EstadDescript(ruta_archivo)
        print(station, minimo, maximo, media, mediana, desvest, varianza)
        resultados.append([station, minimo, maximo, media, mediana, desvest, varianza])
        #if station and minimo: #and maximo and media and mediana and desvest and varianza:
            #resultados.append([station, minimo, maximo, media, mediana, desvest, varianza])
        #else:
            #print(f'Sin suficientes resultado para {archivo}. No se generaron sus estadísticos')
    
    # Se convierten los resultados a un DataFrame
    resultados_df = pd.DataFrame(resultados, columns=['Station', 'minimo', 'maximo', 'media', 'mediana','desvest', 'varianza'])
    
    print(resultados_df)
    
    # Se guarda el DataFrame como un archivo CSV
    resultados_df.to_csv('EstadDescript_TS10_Raw.csv', index=False)

if __name__ == "__main__":
    main()

11105020 -3238.8 3277.8 207.92231389791627 28.0 480.415991217869 230799.52461784758
11135030 -1609.4 1796.4 29.557125021058347 29.9 19.900921659874907 396.0466829124783
12015100 -429.6 3277.8 29.296827333740087 29.1 14.070816732274308 197.98788351325064
12015110 -2892.8 568.1 29.667111931590586 29.4 12.721111368922498 161.82667446052923
13085050 -244.4 3277.8 30.40759005674809 29.9 27.686830742334266 766.5605965546657
15075150 -3251.2 3277.9 33.408308578158746 32.6 51.3839194947984 2640.3071826479227
15085050 -2483.2 5356.4 30.838057976628573 32.0 63.402698455791395 4019.9021714760124
16055120 -524.7 3277.9 24.17295496765101 20.8 51.76886675090377 2680.0155646728294
21015050 -3275.2 3277.8 24.153893007552814 19.3 86.73067865104778 7522.210619271316
21015070 -1636.8 3277.9 18.30573908460283 18.2 22.220359592957724 493.7443804403484
21115010 22.3 3277.8 30.420356886860077 30.0 28.287024654521808 800.1557638055247
21115180 -246.1 3277.8 31.494158179024566 28.8 42.83747124407519 1835.04894

In [15]:
resultados_df = pd.read_csv('EstadDescript_TS10_Raw.csv')

In [12]:
altipatm = pd.read_table('EMA_Patm_TMaxMin.txt', sep=';')

In [13]:
altipatm.head(2)

,OBJECTID,Station,lat,long_,nombre,sede,id_rule,codigo,Instituc,FreqInf,Altitud,TmaxNorm,TminNorm
0,1,11030010.0,5.375,-76.613,Valle,09 - Cali,9,9,IDEAM,H,58,27.853392,19.794912
1,2,11035030.0,5.285,-76.628,Valle,09 - Cali,9,9,IDEAM,H,104,27.376471,19.418962


In [14]:
estadalt = pd.merge(resultados_df, altipatm[['Station','Altitud']], on='Station')

In [16]:
# Se sobreescriben los estadísticos para incluir altitud
estadalt.to_csv('EstadDescript_Patm_Raw.csv', index=False)

In [ ]:
# De datos con QC
def obtener_EstadDescript(archivo):
    try:
        # Se lee el archivo
        datos = pd.read_csv(archivo, encoding='latin-1')#, names=['Fecha', 'Valor'])
        try:
            datos['Fecha'] = pd.to_datetime(datos['Fecha'], format='%Y-%m-%d %H:%M:%S.%f')
        except ValueError:
            datos['Fecha'] = pd.to_datetime(datos['Fecha'], format='%Y-%m-%d %H:%M:%S')
        
        # Se hace el filtro para que solo queden los valores que superaron las pruebas
        dfC = datos[datos['Estado'].apply(lambda x: any([str(x).startswith(prefix) for prefix in ['0PC']]))]
        
        # Se obtiene el código de la estación
        station = dfC['Station'].values[0]
        
        # Se obtienen los estadísticos descriptivos
        minimo = dfC['Valor'].min()
        maximo = dfC['Valor'].max()
        media = dfC['Valor'].mean()
        #mediana = datos['Valor'].median()
        desvest = dfC['Valor'].std()
        varianza = dfC['Valor'].var()
        #first_q = datos['Valor'].quantile(0.25)
        #third_q = datos['Valor'].quantile(0.75)
        
        return station, minimo, maximo, media, desvest, varianza, #first_q, third_q, mediana --no fue posible calcular estos estad.
    
    except Exception as e:
        print(f"Error con el archivo {archivo}: {e}")
        print(e.__class__)
        return None, None, None, None, None, None#, None, None, None --no fue posible calcular estos estad.

def main():
    # Cambia la ruta según la ubicación de tu carpeta
    ruta_carpeta = 'DatosEjemplo'

    # Listamos todos los archivos en la carpeta que terminen con .csv
    archivos = [f for f in os.listdir(ruta_carpeta) if f.endswith('.csv')]
    
    # Creamos una lista para almacenar los resultados
    resultados = []

    # Iteramos sobre cada archivo y obtenemos las fechas
    for archivo in archivos:
        ruta_archivo = os.path.join(ruta_carpeta, archivo)
        station, minimo, maximo, media, desvest, varianza= obtener_EstadDescript(ruta_archivo)
        
        if station and minimo and maximo and media and desvest and varianza:
            resultados.append([station, minimo, maximo, media, desvest, varianza])
    
    # Convertimos los resultados a un DataFrame
    resultados_df = pd.DataFrame(resultados, columns=['Station', 'minimo', 'maximo', 'media', 'desvest', 'varianza'])
    
    # Guardamos el DataFrame como un archivo CSV
    resultados_df.to_csv('EstadDescript_Patm_QC.csv', index=False)

if __name__ == "__main__":
    main()

_____